# Chapter 33: Modified Binary Search

> *"Binary search is not just for finding an element in a sorted array—it's a powerful tool for solving optimization problems and searching in non‑linear spaces."* — Anonymous

---

## 33.1 Introduction to Modified Binary Search

Binary search is one of the most fundamental algorithms, but its true power extends far beyond simple "find x in sorted array" problems. **Modified binary search** refers to adapting the binary search idea to:

- Find elements in partially sorted or rotated arrays.
- Find boundaries (first/last occurrence).
- Solve optimization problems where we search over a range of possible answers (parametric search).
- Compute mathematical functions like square roots.
- Find peaks in arrays.

The common thread is that we have a **monotonic predicate**: some condition is false for a prefix and true for the rest, or vice‑versa. By repeatedly dividing the search space, we can find the transition point efficiently.

### 33.1.1 Why Modified Binary Search Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF MODIFIED BINARY SEARCH               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. EFFICIENCY: Reduces O(n) problems to O(log n).                  │
│  2. VERSATILITY: Works on problems that have a monotonic property.  │
│  3. FOUNDATION: Many classic problems (finding square root,         │
│     aggressive cows, etc.) rely on it.                              │
│  4. INTERVIEW FAVORITE: Rotated array search, peak finding, and     │
│     allocation problems appear frequently.                          │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 33.2 Classic Binary Search Review

Before diving into modifications, recall the standard binary search for a sorted array:

```python
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = left + (right - left) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1
```

All variants maintain the same core idea: maintaining a search interval and narrowing it based on a condition.

---

## 33.3 Searching in Rotated Sorted Array

A rotated sorted array is an array that was sorted and then rotated (shifted) by some pivot. Example: `[4,5,6,7,0,1,2]`. The goal is to search for a target in O(log n) time.

**Key insight:** At least one half of the array is sorted. We can check which half is sorted and decide where to search.

### 33.3.1 Search in Rotated Sorted Array (No Duplicates)

```python
def search_rotated(nums, target):
    left, right = 0, len(nums) - 1
    while left <= right:
        mid = left + (right - left) // 2
        if nums[mid] == target:
            return mid
        # Check if left half is sorted
        if nums[left] <= nums[mid]:
            if nums[left] <= target < nums[mid]:
                right = mid - 1
            else:
                left = mid + 1
        else:  # right half is sorted
            if nums[mid] < target <= nums[right]:
                left = mid + 1
            else:
                right = mid - 1
    return -1
```

### 33.3.2 Search in Rotated Sorted Array with Duplicates

When duplicates are allowed, we may have cases where `nums[left] == nums[mid] == nums[right]` and cannot decide which half is sorted. In that case, we shrink the search space by incrementing `left` and decrementing `right`.

```python
def search_rotated_duplicates(nums, target):
    left, right = 0, len(nums) - 1
    while left <= right:
        mid = left + (right - left) // 2
        if nums[mid] == target:
            return True
        if nums[left] == nums[mid] == nums[right]:
            left += 1
            right -= 1
        elif nums[left] <= nums[mid]:  # left half sorted
            if nums[left] <= target < nums[mid]:
                right = mid - 1
            else:
                left = mid + 1
        else:  # right half sorted
            if nums[mid] < target <= nums[right]:
                left = mid + 1
            else:
                right = mid - 1
    return False
```

### 33.3.3 Find Minimum in Rotated Sorted Array

Another common variant: find the smallest element in a rotated sorted array.

```python
def find_min_rotated(nums):
    left, right = 0, len(nums) - 1
    while left < right:
        mid = left + (right - left) // 2
        if nums[mid] > nums[right]:
            left = mid + 1
        else:
            right = mid
    return nums[left]
```

---

## 33.4 Binary Search on Answer (Parametric Search)

Many optimization problems ask: find the largest (or smallest) value that satisfies a given condition. If the condition is monotonic (i.e., if it's true for x, then it's true for all larger x, or vice‑versa), we can binary search over the possible range of answers.

**General template:**

```python
def condition(x):
    # returns True if x is feasible (or satisfies some property)
    pass

def binary_search_on_answer(lo, hi):
    while lo < hi:
        mid = lo + (hi - lo + 1) // 2  # to avoid infinite loop
        if condition(mid):
            lo = mid
        else:
            hi = mid - 1
    return lo
```

### 33.4.1 Square Root

Find integer square root of a non‑negative integer (LeetCode 69). For any `x`, the condition `mid * mid <= x` is monotonic (true for small mid, false after sqrt).

```python
def my_sqrt(x):
    lo, hi = 0, x
    while lo < hi:
        mid = (lo + hi + 1) // 2
        if mid * mid <= x:
            lo = mid
        else:
            hi = mid - 1
    return lo
```

### 33.4.2 Nth Root

Generalization to k‑th root.

```python
def kth_root(n, k):
    lo, hi = 0, n
    while lo < hi:
        mid = (lo + hi + 1) // 2
        if mid ** k <= n:
            lo = mid
        else:
            hi = mid - 1
    return lo
```

### 33.4.3 Aggressive Cows (Place Cows in Stalls)

**Problem:** Given positions of stalls (sorted) and c cows, place the cows so that the minimum distance between any two cows is maximized.

**Binary search on answer:** Search for the maximum possible minimum distance `d`. Check if we can place all cows with at least `d` distance apart.

```python
def can_place_cows(stalls, cows, dist):
    count = 1
    last = stalls[0]
    for i in range(1, len(stalls)):
        if stalls[i] - last >= dist:
            count += 1
            last = stalls[i]
    return count >= cows

def aggressive_cows(stalls, cows):
    stalls.sort()
    lo, hi = 1, stalls[-1] - stalls[0]
    while lo < hi:
        mid = (lo + hi + 1) // 2
        if can_place_cows(stalls, cows, mid):
            lo = mid
        else:
            hi = mid - 1
    return lo
```

### 33.4.4 Allocation Problems (Book Allocation)

**Problem:** Given an array of pages in books and m students, allocate books such that each student gets a contiguous set, and the maximum pages assigned to any student is minimized.

**Binary search on answer:** Search for the minimum possible maximum pages `limit`. Check if we can partition the books into m contiguous groups each with sum ≤ limit.

```python
def can_allocate(books, students, limit):
    count = 1
    current_sum = 0
    for pages in books:
        if pages > limit:
            return False
        if current_sum + pages > limit:
            count += 1
            current_sum = pages
        else:
            current_sum += pages
    return count <= students

def min_max_pages(books, students):
    lo, hi = max(books), sum(books)
    while lo < hi:
        mid = (lo + hi) // 2
        if can_allocate(books, students, mid):
            hi = mid
        else:
            lo = mid + 1
    return lo
```

### 33.4.5 Koko Eating Bananas (LeetCode 875)

**Problem:** Koko can eat at most `k` bananas per hour; she wants to finish all piles in `h` hours. Find minimum `k`.

```python
def min_eating_speed(piles, h):
    def can_finish(speed):
        hours = 0
        for pile in piles:
            hours += (pile + speed - 1) // speed  # ceil division
        return hours <= h

    lo, hi = 1, max(piles)
    while lo < hi:
        mid = (lo + hi) // 2
        if can_finish(mid):
            hi = mid
        else:
            lo = mid + 1
    return lo
```

---

## 33.5 Finding Peak Element

A peak element is an element that is greater than its neighbors. In an array, we can find any peak in O(log n) using binary search (LeetCode 162).

**Idea:** Look at the middle element. If it is less than its right neighbor, then there must be a peak on the right side; otherwise, the peak is on the left (including mid).

```python
def find_peak_element(nums):
    left, right = 0, len(nums) - 1
    while left < right:
        mid = (left + right) // 2
        if nums[mid] < nums[mid + 1]:
            left = mid + 1
        else:
            right = mid
    return left
```

---

## 33.6 Searching in a 2D Matrix

### 33.6.1 Matrix Sorted Row‑wise and First of Each Row > Last of Previous Row

Treat the matrix as a flattened sorted array and do binary search (LeetCode 74).

```python
def search_matrix(matrix, target):
    if not matrix or not matrix[0]:
        return False
    rows, cols = len(matrix), len(matrix[0])
    lo, hi = 0, rows * cols - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        r, c = mid // cols, mid % cols
        if matrix[r][c] == target:
            return True
        elif matrix[r][c] < target:
            lo = mid + 1
        else:
            hi = mid - 1
    return False
```

### 33.6.2 Matrix with Each Row Sorted but Not Necessarily Global Order

Use binary search per row, or more clever techniques (like searching from top‑right).

---

## 33.7 Binary Search on Real Numbers

When the search space is continuous, we search with a precision `eps` and stop when interval is small enough.

**Example:** Find cube root.

```python
def cube_root(x):
    lo, hi = 0, max(1, x)
    while hi - lo > 1e-6:
        mid = (lo + hi) / 2
        if mid * mid * mid < x:
            lo = mid
        else:
            hi = mid
    return lo
```

---

## 33.8 Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│                    MODIFIED BINARY SEARCH SUMMARY                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Type                      │ Condition / Idea                       │
│────────────────────────────┼────────────────────────────────────────│
│ Rotated Array Search       │ One half is sorted; decide half.       │
│ Find Minimum in Rotated    │ Compare mid with right.                 │
│ Binary Search on Answer    │ Monotonic predicate; search range.     │
│ Peak Finding               │ Move toward the higher neighbor.       │
│ 2D Matrix Search           │ Flatten index or search per row.       │
│ Real‑Number Search         │ Use epsilon and stop condition.        │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 33.9 Practice Problems

### Basic Rotated Array
1. **Search in Rotated Sorted Array** (LeetCode 33)
2. **Search in Rotated Sorted Array II** (LeetCode 81)
3. **Find Minimum in Rotated Sorted Array** (LeetCode 153)
4. **Find Minimum in Rotated Sorted Array II** (LeetCode 154)

### Binary Search on Answer
5. **Sqrt(x)** (LeetCode 69)
6. **Koko Eating Bananas** (LeetCode 875)
7. **Capacity To Ship Packages Within D Days** (LeetCode 1011)
8. **Split Array Largest Sum** (LeetCode 410) – same as book allocation.
9. **Find the Smallest Divisor Given a Threshold** (LeetCode 1283)
10. **Minimum Time to Complete Trips** (LeetCode 2187)
11. **Maximum Candies Allocated to K Children** (LeetCode 2226)

### Peak Finding
12. **Find Peak Element** (LeetCode 162)
13. **Peak Index in a Mountain Array** (LeetCode 852)

### 2D Matrix
14. **Search a 2D Matrix** (LeetCode 74)
15. **Search a 2D Matrix II** (LeetCode 240) – not strictly binary search but uses divide‑and‑conquer.

### Miscellaneous
16. **Find First and Last Position of Element in Sorted Array** (LeetCode 34) – binary search boundaries.
17. **Find K Closest Elements** (LeetCode 658) – binary search to find start.
18. **Median of Two Sorted Arrays** (LeetCode 4) – binary search on partitions.

---

## 33.10 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 12 on binary search trees, but the core binary search ideas are covered.
2. **"Programming Pearls"** by Jon Bentley – Column 4 covers binary search and its applications.
3. **"The Algorithm Design Manual"** by Steven Skiena – Section on binary search.
4. **LeetCode Explore – Binary Search** card.

---

> **Coming in Chapter 34**: **Intervals and Merging** – We'll explore interval problems, merging, and sweep line techniques.

---

**End of Chapter 33**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='32. graph_patterns.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='34. intervals_and_merging.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
